In [13]:
import pandas as pd
import os
import fnmatch
from ast import literal_eval




def find_files(directory, prefix):
    matches = []
    for root, dirnames, filenames in os.walk(directory):
        for filename in fnmatch.filter(filenames, prefix + "*.jsonl"):
            matches.append(os.path.join(root, filename))
    return matches


def get_output(file):
    output = []
    with open(file) as f:
        lines = f.readlines()
    for l in lines:
        l = literal_eval(l)
        output.append(l["response"])
    output = pd.DataFrame(output)
    output.columns = ['response']
    output['personality'] = os.path.splitext(os.path.basename(file))[0].split('_p')[-1][1:].title()
    output['model'] = os.path.basename(file).split('_')[2]
    output['row_id'] = output.index
    
    return output


output_dir = r"D:\71 PsychLLM\outs\personality"
prefix = "model_responses_"

output = pd.DataFrame()
json_files = find_files(output_dir, prefix)
for file in json_files:
    output = pd.concat([output,get_output(file)],axis=0)
output

,response,personality,model,row_id
0,A) Very Accurate,None,falcon-7b-instruct,0
1,A) Very Accurate,None,falcon-7b-instruct,1
2,B) Moderately Accurate,None,falcon-7b-instruct,2
3,A) Very Accurate,None,falcon-7b-instruct,3
4,A) Very Accurate,None,falcon-7b-instruct,4
...,...,...,...,...
115,"Based on the given description, it's likely th...",Task-Specific,zephyr-7b-beta,115
116,"Based on the given description, it seems that ...",Task-Specific,zephyr-7b-beta,116
117,(C). Neither Accurate Nor Inaccurate\n\nExplan...,Task-Specific,zephyr-7b-beta,117
118,(C). Neither Accurate Nor Inaccurate\n\nExplan...,Task-Specific,zephyr-7b-beta,118


In [15]:
output[output['row_id']==3].to_csv('mpi_qualitative.csv')